In [ ]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import backtrader as bt
import backtrader.indicators as btind
import yfinance as yf
import pandas as pd

# --- MACD 策略 ---
class MACDCrossoverFull(bt.Strategy):
    params = (('fast_period', 12), ('slow_period', 26), ('signal_period', 9))

    def __init__(self):
        self.pending_buy = False     # 昨天決定今天買
        self.pending_sell = False    # 昨天決定今天賣
        self.order = None

        self.macd = btind.MACD(period_me1=self.p.fast_period,
                               period_me2=self.p.slow_period,
                               period_signal=self.p.signal_period)
        self.crossover = btind.CrossOver(self.macd.macd, self.macd.signal)

    def notify_order(self, order):
        # Submitted / Accepted → 代表掛單成功，但尚未成交
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status == order.Completed:
            action = "買入" if order.isbuy() else "賣出"
            print(f"{self.data.datetime.date(0)} 📌 {action}成交！"
                  f" 成交價: {order.executed.price:.2f}, "
                  f"股數: {order.executed.size}, "
                  f"現金剩餘: {self.broker.getcash():.2f}")
            self.order = None  # 解鎖

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print(f"{self.data.datetime.date(0)} ⚠️ 訂單失敗（{order.getstatusname()}），已解鎖")
            self.order = None  # 失敗也要解鎖！

    def notify_trade(self, trade):
        if trade.isclosed:
            print(f"{self.data.datetime.date(0)} 🤝 交易平倉：毛利 {trade.pnl:.2f}, 淨利 {trade.pnlcomm:.2f}")

    def next(self):
        # 1️⃣ 若還有訂單等待成交 → 不做事
        if self.order:
            return

        # 2️⃣ 先執行前一天決定的交易 → 用今天開盤價成交
        # 賣在先（避免同一天又買又賣）
        if self.pending_sell and self.position:
            size = self.position.size
            self.order = self.sell(size=size)
            print(f"{self.data.datetime.date(0)} ⛔️ 全部賣出（執行昨日死叉）"
                  f" 開盤價: {self.data.open[0]:.2f}, 股數: {size}")
            self.pending_sell = False
            return  # 今日已執行，退出

        # 再處理買
        if self.pending_buy and not self.position:
            cash = self.broker.getcash()
            open_price = self.data.open[0]
            size = int(cash // open_price)

            if size > 0:
                self.order = self.buy(size=size)
                print(f"{self.data.datetime.date(0)} 🚀 全倉買進（執行昨日金叉）"
                      f" 開盤價: {open_price:.2f}, 股數: {size}")
            else:
                print(f"{self.data.datetime.date(0)} ⚠️ 現金不足無法買進")

            self.pending_buy = False
            return

        # 3️⃣ 使用今日收盤資料判斷明日行動
        if self.crossover[0] > 0:
            self.pending_buy = True     # 金叉 → 明天買
            self.pending_sell = False
            # print(f"{self.data.datetime.date(0)} ▶️ 金叉出現，預約明天買")
        elif self.crossover[0] < 0:
            self.pending_sell = True    # 死叉 → 明天賣
            self.pending_buy = False
            # print(f"{self.data.datetime.date(0)} ◀️ 死叉出現，預約明天賣")



# --- Buy & Hold 策略 (與 Sizer 配合) ---
class BuyAndHoldFull(bt.Strategy):
    def __init__(self):
        self.buy_signal = False  # 當天觸發購買信號
        self.order = None

    def next(self):
        # 如果還沒買，觸發購買信號
        if not self.position and not self.buy_signal:
            self.buy_signal = True
            return

        # 前一天觸發 buy_signal → 今天開盤執行
        if self.buy_signal and not self.position:
            open_price = self.data.open[0]
            cash = self.broker.getcash()

            size = int(cash // open_price)

            if size > 0:
                self.order = self.buy(size=size)
                print(f"{self.data.datetime.date(0)} 🚀 全倉買進！價格: {open_price:.2f}, 股數: {size}")
            else:
                print(f"{self.data.datetime.date(0)} ⚠️ 現金不足，無法買入")

            self.buy_signal = False

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                print(f"{self.data.datetime.date(0)} 📌 成交！成交價: {order.executed.price:.2f}, "
                      f"股數: {order.executed.size}")

# MACD + 布林通道策略                
class MACD_BB_Full(bt.Strategy):
    params = dict(
        fast_period=12,
        slow_period=26,
        signal_period=9,
        bb_period=20,
        bb_dev=2,
    )

    def __init__(self):
        self.order = None
        self.pending_buy = False
        self.pending_sell = False

        # MACD 指標
        self.macd = btind.MACD(
            self.data,
            period_me1=self.p.fast_period,
            period_me2=self.p.slow_period,
            period_signal=self.p.signal_period
        )
        self.crossover = btind.CrossOver(self.macd.macd, self.macd.signal)

        # 布林通道
        self.bbands = btind.BollingerBands(
            self.data.close,
            period=self.p.bb_period,
            devfactor=self.p.bb_dev
        )

    # 訂單狀態紀錄
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status == order.Completed:
            action = "買入" if order.isbuy() else "賣出"
            print(f"{self.data.datetime.date(0)} 📌 {action}成交！"
                  f" 成交價: {order.executed.price:.2f}, "
                  f"股數: {order.executed.size}, "
                  f"現金剩餘: {self.broker.getcash():.2f}")
            self.order = None

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            print(f"{self.data.datetime.date(0)} ⚠️ 訂單失敗（{order.getstatusname()}）")
            self.order = None

    def notify_trade(self, trade):
        if trade.isclosed:
            print(f"{self.data.datetime.date(0)} 🤝 平倉：毛利 {trade.pnl:.2f}")

    def next(self):
        if self.order:
            return

        close = self.data.close[0]
        openp = self.data.open[0]
        mid = self.bbands.mid[0]
        top = self.bbands.top[0]
        bot = self.bbands.bot[0]

        # === Step1: 先執行昨天決定的動作 (用今天開盤價) ===
        if self.pending_sell and self.position:
            size = self.position.size
            self.order = self.sell(size=size)
            print(f"{self.data.datetime.date(0)} ⛔️ 全部賣出（執行昨日死叉） 開盤: {openp:.2f}")
            self.pending_sell = False
            return

        if self.pending_buy and not self.position:
            cash = self.broker.getcash()
            size = int(cash // openp)

            if size > 0:
                self.order = self.buy(size=size)
                print(f"{self.data.datetime.date(0)} 🚀 全倉買進（執行昨日金叉+布林）"
                      f" 開盤: {openp:.2f} 股數: {size}")
            else:
                print(f"{self.data.datetime.date(0)} ⚠️ 現金不足無法買進")
            self.pending_buy = False
            return

        # === Step2: 用今天收盤判斷明天行動 ===
        if self.crossover[0] > 0:  # 金叉 → 偏多
            if close <= mid:  # 價格在布林通道中軌下側才買
                self.pending_buy = True
                self.pending_sell = False

        elif self.crossover[0] < 0:  # 死叉 → 偏空
            self.pending_sell = True
            self.pending_buy = False

# --- MACD + 布林 + ADX 策略 ---
class MACD_BB_ADX_Full(bt.Strategy):
    params = dict(
        fast_period=12,
        slow_period=26,
        signal_period=9,
        bb_period=20,
        bb_dev=2,
        adx_period=14,
        adx_threshold=30,
    )

    def __init__(self):
        self.order = None
        self.pending_buy = False
        self.pending_sell = False
        self.count = 0

        # MACD
        self.macd = btind.MACD(self.data,
                               period_me1=self.p.fast_period,
                               period_me2=self.p.slow_period,
                               period_signal=self.p.signal_period)
        self.crossover = btind.CrossOver(self.macd.macd, self.macd.signal)

        # 布林通道
        self.bbands = btind.BollingerBands(self.data.close,
                                           period=self.p.bb_period,
                                           devfactor=self.p.bb_dev)

        # ADX + DI
        self.adx = btind.ADX(self.data, period=self.p.adx_period)
        self.di_pos = btind.PlusDI(self.data, period=self.p.adx_period)
        self.di_neg = btind.MinusDI(self.data, period=self.p.adx_period)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            if order.status == order.Accepted:
                print(f"{self.data.datetime.date(0)} 📌 訂單已接受")
            elif order.status == order.Submitted:
                print(f"{self.data.datetime.date(0)} 📌 訂單已提交")
            return
        if order.status == order.Completed:
            action = "買入" if order.isbuy() else "賣出"
            print(f"{self.data.datetime.date(0)} 📌 {action}成交！"
                  f" 價格: {order.executed.price:.2f}, 股數: {order.executed.size}")
            if not order.isbuy():
                self.count += 1
            self.order = None
        else:
            print(f"{self.data.datetime.date(0)} ⚠️ 訂單失敗 ({order.getstatusname()})")
            self.order = None

    def next(self):
        if self.order:
            return

        openp = self.data.open[0]
        close = self.data.close[0]
        mid = self.bbands.mid[0]
        adx_val = self.adx[0]
        dip = self.di_pos[0]
        din = self.di_neg[0]

        # Step1: 執行前一天計劃
        if self.pending_sell and self.position:
            size = self.position.size
            self.order = self.sell(size=size)
            print(f"{self.data.datetime.date(0)} ⛔️ 賣出（執行昨日死叉+DI+ADX）")
            self.pending_sell = False
            return

        if self.pending_buy and not self.position:
            cash = self.broker.getcash()
            size = int(cash // openp)
            if size > 0:
                self.order = self.buy(size=size)
                print(f"{self.data.datetime.date(0)} 🚀 買進（執行昨日金叉+DI+BB+ADX）")
            self.pending_buy = False
            return

        # 趨勢過濾：ADX 必須夠強 (ADX > 30)
        if adx_val > self.p.adx_threshold:
            
            # === 買入訊號判斷 ===
            # 1. 觸發條件: MACD金叉
            if self.crossover[0] > 0:
                # 2. 過濾條件: DI線確認為多頭主導，且價格相對偏低
                if close <= mid: # ⭐️ 移除 DI+ > DI- 的硬性要求 ⭐️
                    self.pending_buy = True
                    self.pending_sell = False
                    print(f"{self.data.datetime.date(0)} 📈 買入訊號: MACD金叉 + ADX強 + BB中線以下")
            
            # === 賣出訊號判斷 ===
            elif self.crossover[0] < 0:
                # 2. 過濾條件: 移除 DI- > DI+ 的硬性要求 
                self.pending_sell = True
                self.pending_buy = False
                print(f"{self.data.datetime.date(0)} 📉 賣出訊號: MACD死叉 + ADX強")


# --- 下載資料 (加上 auto_adjust=True 確保使用調整後價格) ---
# 用同一隻股票比較策略跟 Buy & Hold 的差異
symbol = '2330'
STOCK_TICKER = f'{symbol}.TW'
BENCH_TICKER = f'{symbol}.TW'
START = '2020-01-01'
END = ''

# **重要修正：加上 auto_adjust=True 確保使用調整後價格**
df2330 = yf.download(STOCK_TICKER, start=START, end=END, auto_adjust=True)
df0050 = yf.download(BENCH_TICKER, start=START, end=END, auto_adjust=True)

# 如果 auto_adjust=True，通常不會有多級索引，但保留程式碼以防萬一
if isinstance(df2330.columns, pd.MultiIndex):
    df2330.columns = df2330.columns.droplevel(1)

if isinstance(df0050.columns, pd.MultiIndex):
    df0050.columns = df0050.columns.droplevel(1)


# --- 回測 MACD 策略 ---
data1 = bt.feeds.PandasData(dataname=df2330)
cerebro1 = bt.Cerebro()
cerebro1.addstrategy(MACDCrossoverFull)
cerebro1.adddata(data1)
cerebro1.broker.setcash(100000)
print("\n🚀 回測 MACD 策略中 ...")
cerebro1.run()
macd_final_value = cerebro1.broker.getvalue()
print("===========================")
# --- 回測 Buy & Hold 策略 ---
data2 = bt.feeds.PandasData(dataname=df0050)
cerebro2 = bt.Cerebro()
cerebro2.addsizer(bt.sizers.FixedSize, stake=100)
cerebro2.addstrategy(BuyAndHoldFull)
cerebro2.adddata(data2)
cerebro2.broker.setcash(100000)
print("\n📌 回測 Buy & Hold 策略中 ...")
cerebro2.run()
bh_final_value = cerebro2.broker.getvalue()
# --- 回測 MACD + 布林通道策略 ---
data3 = bt.feeds.PandasData(dataname=df2330)
cerebro3 = bt.Cerebro()
cerebro3.addstrategy(MACD_BB_Full)
cerebro3.adddata(data3)
cerebro3.broker.setcash(100000)
print("\n📌 回測 MACD + 布林通道策略中 ...")
cerebro3.run()
bb_final_value = cerebro3.broker.getvalue()
# --- 回測 MACD + BB + ADX 策略 ---
data4 = bt.feeds.PandasData(dataname=df2330)
cerebro4 = bt.Cerebro()
cerebro4.addstrategy(MACD_BB_ADX_Full)
cerebro4.adddata(data4)
cerebro4.broker.setcash(100000)

# 加上 Analyzer
cerebro4.addanalyzer(bt.analyzers.DrawDown)
cerebro4.addanalyzer(bt.analyzers.TradeAnalyzer)

print("\n⚡ 回測 MACD + BB + ADX 策略中 ...")
result4 = cerebro4.run()
mba_final_value = cerebro4.broker.getvalue()


# --- 結果輸出 ---
print("\n===========================")
print("📊 回測結果比較")
print(f"期間: {START} 至 {END}")
print("---------------------------")
print(f"B&H(Benchmark) ({BENCH_TICKER}) 最終資金: {bh_final_value:.2f} ─ 報酬率: {(bh_final_value/100000-1):.2%}")
print(f"MACD ({STOCK_TICKER}) 最終資金: {macd_final_value:.2f}  ─ 報酬率: {(macd_final_value/100000-1):.2%}")
print(f"MACD + BB ({STOCK_TICKER}) 最終資金: {bb_final_value:.2f} ─ 報酬率: {(bb_final_value/100000-1):.2%}")
print(f"MACD + BB + ADX ({STOCK_TICKER}) 最終資金: {mba_final_value:.2f} ─ 報酬率: {(mba_final_value/100000-1):.2%}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



🚀 回測 MACD 策略中 ...
2020-03-27 🚀 全倉買進（執行昨日金叉） 開盤價: 254.91, 股數: 392
2020-03-30 📌 買入成交！ 成交價: 236.51, 股數: 392, 現金剩餘: 7287.77
2020-05-13 ⛔️ 全部賣出（執行昨日死叉） 開盤價: 262.99, 股數: 392
2020-05-14 📌 賣出成交！ 成交價: 262.99, 股數: -392, 現金剩餘: 110379.55
2020-05-14 🤝 交易平倉：毛利 10379.55, 淨利 10379.55
2020-06-04 🚀 全倉買進（執行昨日金叉） 開盤價: 273.76, 股數: 403
2020-06-05 ⚠️ 訂單失敗（Margin），已解鎖
2020-06-29 🚀 全倉買進（執行昨日金叉） 開盤價: 284.09, 股數: 388
2020-06-30 📌 買入成交！ 成交價: 283.64, 股數: 388, 現金剩餘: 326.97
2020-08-12 ⛔️ 全部賣出（執行昨日死叉） 開盤價: 381.81, 股數: 388
2020-08-13 📌 賣出成交！ 成交價: 387.24, 股數: -388, 現金剩餘: 150574.18
2020-08-13 🤝 交易平倉：毛利 40194.63, 淨利 40194.63
2020-09-17 🚀 全倉買進（執行昨日金叉） 開盤價: 412.10, 股數: 365
2020-09-18 📌 買入成交！ 成交價: 406.65, 股數: 365, 現金剩餘: 2148.71
2020-09-22 ⛔️ 全部賣出（執行昨日死叉） 開盤價: 400.28, 股數: 365
2020-09-23 📌 賣出成交！ 成交價: 396.64, 股數: -365, 現金剩餘: 146921.65
2020-09-23 🤝 交易平倉：毛利 -3652.53, 淨利 -3652.53
2020-10-12 🚀 全倉買進（執行昨日金叉） 開盤價: 416.65, 股數: 352
2020-10-13 ⚠️ 訂單失敗（Margin），已解鎖
2020-11-10 🚀 全倉買進（執行昨日金叉） 開盤價: 411.19, 股數: 357
2020-11-11 📌 買入成交！ 成交價: 40

AttributeError: 'NoneType' object has no attribute '_next_stid'

In [13]:
trade_analysis = result4[0].analyzers.tradeanalyzer.get_analysis()
win_rate = trade_analysis.won.total / trade_analysis.total.total
print(f"勝率: {win_rate:.2%}")


勝率: 100.00%
